In [1]:
# Importing the necessary libraries required for the working
import pytesseract
from pdf2image import convert_from_path
from pytesseract import image_to_string
from sklearn.feature_extraction import text
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
import re
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
import string

import warnings
warnings.simplefilter('ignore')

C:\Users\bikem\anaconda3\envs\nlp_course\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# defining the functions to extract the text from pdf using the above mentioned libraries
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
def convert_pdf_to_img(pdf_file):
    return convert_from_path(pdf_file)

def convert_image_to_string(file):
    text = image_to_string(file)
    return text

def get_text_from_pdf_file(pdf_file):
    images = convert_pdf_to_img(pdf_file)
    final_text = ""
    for pg, img in enumerate(images):
        final_text += convert_image_to_string(img)
        
    return final_text

In [3]:
# extracting the text
text1 = get_text_from_pdf_file("Kerala State Beverages vs Assistant State Commissioner.pdf")

text2 = get_text_from_pdf_file("CASE_1.pdf")

text3 = get_text_from_pdf_file("Devarajan Raman vs Bank of India.pdf")

In [6]:
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

# Based on the extracted text, we now extract the words under the text from the pdf document post cleaning, 
# removing the stopwords and cleaning the text from unwanted materials
def extract_from_text_unique_words(text):
    words = ' '.join([word for word in text.lower().split() if word not in STOPWORDS])
    words = ''.join(ch for ch in words if ch not in exclude)
    words = ' '.join([lemma.lemmatize(word) for word in words.split()])
    #new_text1 = " ".join(words)
    new_text1 = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", words)
    new_text1 = "".join(filter(lambda x: not x.isdigit(), new_text1))
    wtok1 = word_tokenize(new_text1)
    words = pd.DataFrame(wtok1)
    new_words1 = list(dict.fromkeys(wtok1))
    words.rename(columns={0 :'words1'}, inplace=True)
    words = words['words1'].unique()
    words1 = pd.DataFrame(words)
    words1.rename(columns={0:'words'}, inplace=True)
    return words1

In [7]:
words_1 = extract_from_text_unique_words(text1)
words_2 = extract_from_text_unique_words(text2)
words_3 = extract_from_text_unique_words(text3)

In [8]:
words_1

,words
0,caslpcno
1,etc
2,reportable
3,supreme
4,court
...,...
778,hrishikesh
779,roy
780,new
781,delhi


In [9]:
words_2

,words
0,reportable
1,supreme
2,court
3,india
4,criminal
...,...
472,ashok
473,bhushan
474,eae
475,ceeeeeneeseseaeeneneeees


In [10]:
words_3

,words
0,reportable
1,supreme
2,court
3,india
4,civil
...,...
449,kumari
450,gaur
451,arcumps
452,master


In [11]:
words_merged = pd.merge(words_1,words_2,on='words', how='outer')
words_merged = words_merged.sort_values('words', ascending=True)
words_merged

,words
153,a
315,abkari
772,above
295,abovesaid
1048,absence
...,...
325,word
326,worded
971,wrong
58,year


In [12]:
wrds = pd.merge(words_merged,words_3, on='words', how='outer')
wrds = wrds.sort_values('words', ascending=True)
wrds

,words
0,a
1299,abdication
1,abkari
2,above
1184,abovementioned
...,...
1082,wrong
1323,xvii
1089,y
1083,year
